# Script Match DataFrames

## Importo

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from difflib import SequenceMatcher
import os
import unicodedata
import re
from deep_translator import GoogleTranslator

## Cargo dataframes

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# Variable para manejar si se trabaja en local o en Fabric
local = True

# Rutas de los archivos
if local:
    # Ruta local
    ruta_csv = os.path.join('..', 'Ke30', 'Ke30 Original 202501.csv')
    ruta_excel = os.path.join('..', 'Ke30', 'Ke30  202501.xlsx')
else:
    ruta_csv = '/lakehouse/default/Files/Ke30 Original 202501.csv'
    ruta_excel = '/lakehouse/default/Files/Ke30  202501.xlsx'

# Cargo los archivos como DataFrames
df_original = pd.read_csv(ruta_csv, encoding='latin1', sep=';')
df_sap = pd.read_excel(ruta_excel)

C:\Users\maximo.lloris\AppData\Local\Temp\ipykernel_15752\2657139154.py:14: DtypeWarning: Columns (3,10,11,13,14,16,20,42,144,146,148,150,152) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv(ruta_csv, encoding='latin1', sep=';')


## Añado índice

In [4]:
# Creo identificador único concatenando las columnas
df_original['ID_unico_calculado'] = (
    df_original['Organización ventas'].astype(str) + '-' +
    df_original['Centro de beneficio'].astype(str) + '-' +
    df_original['Organiz.ventas'].astype(str)
)

# Creo identificador único concatenando las columnas
df_sap['ID_unico_calculado'] = (
    df_sap['COMPANY_CODE_ID'].astype(str) + '-' +
    df_sap['CEPCT_PRACTICE_ID'].astype(str) + '-' +
    df_sap['SALES_OFFICE_TEXT'].astype(str)
)

# Reordeno columnas en df_original
df_original.insert(0, 'ID_unico_calculado', df_original.pop('ID_unico_calculado'))

# Reordeno columnas en df_sap
df_sap.insert(0, 'ID_unico_calculado', df_sap.pop('ID_unico_calculado'))

# Ordeno los DataFrames por ID_unico_calculado de menor a mayor
# df_original.sort_values(by='ID_unico_calculado', inplace=True)
# df_sap.sort_values(by='ID_unico_calculado', inplace=True)

# Reinicio el índice después del ordenamiento para mantener el DataFrame limpio
# df_original.reset_index(drop=True, inplace=True)
# df_sap.reset_index(drop=True, inplace=True)

In [4]:
def convertir_a_float_si_posible(columna):
    """
    Intenta convertir una columna a float solo si es de tipo object o string.
    - No modifica columnas de tipo int64.
    - Elimina separadores de miles (puntos) y reemplaza comas por puntos antes de la conversión.
    - Si la conversión falla, devuelve la columna original.
    """
    if columna.dtype in ['int64', 'float64']:  # No modifica las columnas numericas
        return columna

    try:
        return (
            columna.astype(str)
            .apply(lambda x: x.replace('.', '') if (',' in x and '.' in x) else x)  # Elimino separadores de miles si existe tambien una coma
            .str.replace(',', '.', regex=False)  # Reemplazo comas por puntos decimales
            .astype(float)  # Convierto a float
        )
    except ValueError:
        return columna  # Devuelve la columna original si no puede convertirse

In [5]:
# Aplicar la conversión solo en columnas que no sean enteros
df_original = df_original.apply(convertir_a_float_si_posible)
df_sap = df_sap.apply(convertir_a_float_si_posible)


## Preprocesado

In [6]:
def preprocesar_dataframe(columna, nombre_columna):
    """
    Castea los valores de una columna en el orden: int -> float -> str.
    Maneja valores nulos de forma segura e imprime el tipo de dato asignado.
    """
    try:
        # Intento de conversión a int
        columna_convertida = pd.to_numeric(columna, errors='raise').astype('Int64')  # Int64 permite NaN
        print(f"La columna '{nombre_columna}' es Int64. Se aplicará macheo por moda.")
        return columna_convertida
    except (ValueError, TypeError):
        try:
            # Intento de conversión a float
            columna_convertida = pd.to_numeric(columna, errors='raise').astype(float)
            print(f"La columna '{nombre_columna}' es float64. Se aplicará macheo por moda.")
            return columna_convertida
        except (ValueError, TypeError):
            # Conversión a string
            print(f"La columna '{nombre_columna}' es object (string). Se aplicará macheo por moda o valores.")
            return columna.astype(str)


## Modas V2 (20 modas)

In [7]:
def similitud(a, b):
    """
    Calcula la similitud entre dos valores usando SequenceMatcher.
    Devuelve 0 si alguno de los valores es nulo.
    """
    if pd.isnull(a) or pd.isnull(b):
        return 0
    return SequenceMatcher(None, str(a), str(b)).ratio()

def obtener_modas(columna, num_modas=20, min_repeticiones_moda=5):
    """
    Obtiene hasta 'num_modas' valores más frecuentes de una columna,
    pero solo si tienen al menos 'min_repeticiones_moda' repeticiones.
    """
    conteos = columna.value_counts()
    modas_filtradas = conteos[conteos >= min_repeticiones_moda].index.tolist()
    return modas_filtradas[:num_modas]  # Retornar hasta 'num_modas' modas

def comparar_modas(columna_df1, df2, num_modas=20, umbral_similitud=1.0, 
                   min_coincidencias=3, min_repeticiones_moda=5):
    """
    Compara hasta 'num_modas' modas de una columna de df1 con las de df2.
    Un macheo solo es válido si al menos 'min_coincidencias' modas coinciden.
    """
    modas_df1 = obtener_modas(columna_df1, num_modas, min_repeticiones_moda)
    mejor_columna = None
    mejor_coincidencias = 0

    # Precalculo las modas de todas las columnas de df2
    modas_df2_dict = {col: obtener_modas(df2[col], num_modas, min_repeticiones_moda) for col in df2.columns}

    # Itero sobre columnas precalculadas
    for columna, modas_df2 in modas_df2_dict.items():
        # Cuento el número de coincidencias exactas entre las modas
        coincidencias = sum(
            any(similitud(moda1, moda2) >= umbral_similitud for moda2 in modas_df2)
            for moda1 in modas_df1
        )

        # Solo considero macheo si hay al menos 'min_coincidencias'
        if coincidencias >= min_coincidencias and coincidencias > mejor_coincidencias:
            mejor_columna = columna
            mejor_coincidencias = coincidencias

            # Macheo perfecto si todas las modas coinciden
            if coincidencias == num_modas:
                print(f"✅ Macheo perfecto de modas ({num_modas}/{num_modas} coincidencias) con la columna '{columna}'.")
                break

    if mejor_columna:
        print(f"🔗 Mejor macheo por modas para la columna '{columna_df1.name}' es '{mejor_columna}' con {mejor_coincidencias}/{num_modas} coincidencias.")

    return mejor_columna if mejor_coincidencias >= min_coincidencias else None


## Macheo de valores

In [9]:
def similitud_valores(a, b):
    """
    Calcula la similitud entre dos valores usando SequenceMatcher.
    Devuelve 0 si alguno de los valores es nulo.
    """
    if pd.isnull(a) or pd.isnull(b):
        return 0
    return SequenceMatcher(None, str(a).lower(), str(b).lower()).ratio()


def obtener_modas_valores(columna, num_modas=10, tipo='string'):
    """
    Obtiene hasta 'num_modas' valores más frecuentes (modas) de una columna.
    Filtra según el tipo: 
    - Para strings: excluye nulos.
    - Para numéricos: excluye 0.00, ceros consecutivos, nulos.
    """
    if tipo == 'string':
        columna_filtrada = columna.dropna()
    else:  # tipo numérico
        return []  # No aplica para valores numéricos
        #columna_filtrada = columna[(~columna.isnull()) & (columna != 0) & (~columna.astype(str).str.fullmatch(r'0+'))]

    return columna_filtrada.value_counts().index.tolist()[:num_modas]


def macheo_por_valores(columna_df1, df2, num_valores=10):
    """
    Compara hasta 'num_valores' modas de la columna de df1 con los valores únicos de columnas de df2.
    - Para strings: al menos 2 coincidencias con umbral de similitud >= 0.7.
    - Para numéricos: al menos 7 coincidencias con umbral de similitud >= 0.7.
    Antes de aplicar el criterio, castea los valores a float para evitar discrepancias.
    """
    # Castear valores a float si es posible
    try:
        columna_df1 = columna_df1.astype(str).str.replace(',', '.', regex=False).astype(float)
        df2 = df2.apply(lambda x: x.astype(str).str.replace(',', '.', regex=False).astype(float) if x.dtype != object else x)
        tipo_dato = 'numeric'
    except ValueError:
        tipo_dato = 'string'

    umbral_similitud = 0.7
    coincidencias_requeridas = 2 if tipo_dato == 'string' else 7

    modas_df1 = obtener_modas_valores(columna_df1, num_valores, tipo=tipo_dato)
    

    mejor_columna = None
    mejor_coincidencia = 0
    detalles_coincidencias = {}

    for columna in df2.columns:
        valores_df2 = df2[columna].dropna().unique()[:num_valores]
        

        coincidencias = 0
        detalles_coincidencias[columna] = []

        for moda in modas_df1:
            for valor in valores_df2:
                sim = similitud_valores(moda, valor)
                if sim >= umbral_similitud:
                    coincidencias += 1
                    detalles_coincidencias[columna].append(
                        f"✅ '{moda}' coincide con '{valor}' (similitud: {round(sim, 2)})"
                    )
                    break
            else:
                detalles_coincidencias[columna].append(f"❌ '{moda}' no encontró coincidencia adecuada.")

        if coincidencias >= coincidencias_requeridas and coincidencias > mejor_coincidencia:
            mejor_coincidencia = coincidencias
            mejor_columna = columna

        if mejor_coincidencia >= coincidencias_requeridas:
            break

    if mejor_columna:
        print(f"\n🔗 Macheo exitoso por valores con la columna '{mejor_columna}' "
              f"({mejor_coincidencia}/{len(modas_df1)} coincidencias, tipo {tipo_dato}).\n")
        print("📝 Detalles de las coincidencias encontradas:")
        for detalle in detalles_coincidencias[mejor_columna]:
            print(f" - {detalle}")

    return mejor_columna

## Similitud nombres columnas

In [10]:
def estandarizar_texto(texto):
    """
    Estandariza el texto: minúsculas, elimina acentos, caracteres especiales, 
    reemplaza 'ñ' por 'n' y espacios por guiones bajos.
    """
    texto = str(texto).lower().strip()
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = texto.replace('ñ', 'n')
    texto = texto.replace(' ', '_')
    texto = re.sub(r'[^\w_]', '', texto)
    return texto

def similitud_cadenas(a, b):
    """
    Calcula la similitud entre dos strings utilizando SequenceMatcher después de estandarizar el texto.
    """
    a = estandarizar_texto(a)
    b = estandarizar_texto(b)
    return SequenceMatcher(None, a, b).ratio()

def buscar_similitud_nombres(nombre_columna_df1, df2, umbral=0.75):
    """
    Busca la columna de df2 cuyo nombre sea más similar al nombre proporcionado ('nombre_columna_df1')
    utilizando la similitud de texto estandarizado.
    """
    mejor_similitud = 0
    mejor_columna = None

    for columna in df2.columns:
        sim = similitud_cadenas(nombre_columna_df1, columna)
        if sim > mejor_similitud and sim >= umbral:
            mejor_similitud = sim
            mejor_columna = columna

    if mejor_columna:
        print(f"✅ Mejor coincidencia de nombres de columna para '{nombre_columna_df1}' es '{mejor_columna}' con una similitud del {round(mejor_similitud * 100, 2)}%.")

    return mejor_columna

## Traduccion nombres

In [18]:
def estandarizar_texto(texto):
    """
    Estandariza el texto: minúsculas, elimina acentos, caracteres especiales, 
    reemplaza 'ñ' por 'n' y espacios por guiones bajos.
    """
    if not texto or not isinstance(texto, str):
        return ""
    
    texto = texto.lower().strip()
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')  # Eliminar acentos
    texto = texto.replace('ñ', 'n')
    texto = texto.replace(' ', '_')
    texto = re.sub(r'[^\w_]', '', texto)  # Eliminar caracteres especiales excepto "_"
    
    return texto

def similitud_cadenas(a, b):
    """
    Calcula la similitud entre dos strings utilizando SequenceMatcher después de estandarizar el texto.
    """
    a = estandarizar_texto(a)
    b = estandarizar_texto(b)
    return SequenceMatcher(None, a, b).ratio()

def traducir_nombres_columnas(nombres_columnas):
    """
    Traduce una lista de nombres de columnas de español a inglés usando Deep Translator.
    Devuelve un diccionario {nombre_original: nombre_traducido}.
    """
    if not nombres_columnas:
        return {}

    traducciones = {}
    try:
        for nombre in nombres_columnas:
            traducciones[nombre] = GoogleTranslator(source="es", target="en").translate(nombre)
    except Exception as e:
        print(f" Error en la traducción: {e}")
        traducciones = {nombre: nombre for nombre in nombres_columnas}  # Devuelve los nombres originales en caso de error

    return traducciones


def buscar_similitud_nombres(nombre_columna_df1, df2, traducciones, umbral=0.75):
    """
    Busca la columna de df2 cuyo nombre sea más similar al nombre traducido de df1.
    - Usa el diccionario de traducciones para machear contra df2.
    """
    mejor_similitud = 0
    mejor_columna = None

    # Obtener el nombre traducido
    nombre_traducido = traducciones.get(nombre_columna_df1, nombre_columna_df1)

    for columna in df2.columns:
        sim = similitud_cadenas(nombre_traducido, columna)
        if sim > mejor_similitud and sim >= umbral:
            mejor_similitud = sim
            mejor_columna = columna

    if mejor_columna:
        print(f"✅ Mejor coincidencia para '{nombre_columna_df1}' ('{nombre_traducido}') → '{mejor_columna}' ({round(mejor_similitud * 100, 2)}%)")

    return mejor_columna


## xx_Ejecutar traducciones (evitar ejecutar esta celda muchas veces para evitar llamadas a la API)

In [16]:
# Crear ejemplos simples para probar la función
df1 = df_original
df2 = df_sap

In [21]:
# Obtengo los nombres originales de df1
nombres_originales = list(df1.columns)

# Traduzco las columnas de df1 a inglés en una sola llamada
traducciones = traducir_nombres_columnas(nombres_originales)

## Funcion principal

In [23]:
def macheo_columnas_df(df1, df2, utilizar_columnas_ya_utilizadas=False, realizar_preprocesado=False):
    """
    Realiza el macheo de columnas entre dos DataFrames aplicando los siguientes criterios en orden:
    1. Coincidencia entre las modas y similitud en el nombre de la columna.
    2. Coincidencia por modas.
    3. Coincidencia por valores.
    4. Coincidencia por similitud en el nombre de la columna.
    
    Devuelve el DataFrame fusionado, columnas sin correlación, columnas no utilizadas, columnas utilizadas múltiples veces,
    porcentaje de éxito, el número de coincidencias por cada criterio y un DataFrame de mapeo de columnas.
    """
    # Inicialización
    df_fusionado = pd.DataFrame()
    df_no_correlacionado = pd.DataFrame()
    df_mapeo_fusion = pd.DataFrame(columns=["ORIGINAL", "SAP"])
    columnas_no_utilizadas = list(df2.columns)
    columnas_utilizadas = set()  # Cambiado a conjunto para mejorar eficiencia
    columnas_utilizadas_multiples_veces = {}
    contador_correlacion = 1
    macheo_exitoso = 0
    mode_col_name_matching = 0
    mode_matching = 0
    values_matching = 0
    similarity_matching = 0

    if realizar_preprocesado:
        for col in df1.columns:
            df1[col] = preprocesar_dataframe(df1[col], col)
        for col in df2.columns:
            df2[col] = preprocesar_dataframe(df2[col], col)

    # Iterar sobre columnas de df1
    for columna in df1.columns:
        print(f"🔍 Analizando columna de df1: '{columna}'")
        columna_en_df2 = None
        criterio_utilizado = None
        columnas_disponibles = [col for col in df2.columns if utilizar_columnas_ya_utilizadas or col not in columnas_utilizadas]

        # Criterio combinado: modas + similitud en el nombre
        for col_sap in columnas_disponibles:
            if (buscar_similitud_nombres(columna, pd.DataFrame({col_sap: df2[col_sap]}), traducciones) == col_sap and
                comparar_modas(df1[columna], df2[[col_sap]]) == col_sap):
                columna_en_df2 = col_sap
                criterio_utilizado = "mode_col_name"
                mode_col_name_matching += 1
                break
            

        # Macheo por moda
        if not columna_en_df2:
            for col_sap in columnas_disponibles:
                if comparar_modas(df1[columna], df2[[col_sap]]) == col_sap:
                    columna_en_df2 = col_sap
                    criterio_utilizado = "mode"
                    mode_matching += 1
                    break

        # Macheo por valores
        if not columna_en_df2:
            for col_sap in columnas_disponibles:
                if macheo_por_valores(df1[columna], df2[[col_sap]]) == col_sap:
                    columna_en_df2 = col_sap
                    criterio_utilizado = "values"
                    values_matching += 1
                    break
                

        # Similitud de nombres
        if not columna_en_df2:
            for col_sap in columnas_disponibles:
                if buscar_similitud_nombres(columna, pd.DataFrame({col_sap: df2[col_sap]}), traducciones) == col_sap:
                    columna_en_df2 = col_sap
                    criterio_utilizado = "similarity_col_name"
                    similarity_matching += 1
                    break

        # Si se encuentra coincidencia
        if columna_en_df2:
            macheo_exitoso += 1
            nuevo_nombre = f"{columna_en_df2}_{criterio_utilizado}"
            df_fusionado = pd.concat(
                [df_fusionado, df1[[columna]], df2[[columna_en_df2]].rename(columns={columna_en_df2: nuevo_nombre})],
                axis=1
            )

            # Actualizar registros de columnas utilizadas
            if columna_en_df2 in columnas_utilizadas:
                columnas_utilizadas_multiples_veces[columna_en_df2] = columnas_utilizadas_multiples_veces.get(columna_en_df2, 0) + 1
            columnas_utilizadas.add(columna_en_df2)

        else:
            # Si no se encuentra ningún macheo
            columna_en_df2 = f"buscar_correlacion_sap_{contador_correlacion}"
            df_no_correlacionado = pd.concat(
                [df_no_correlacionado, df1[[columna]]], axis=1
            )
            df_fusionado = pd.concat(
                [
                    df_fusionado,
                    df1[[columna]],
                    pd.DataFrame({columna_en_df2: [np.nan] * len(df1)})
                ],
                axis=1,
            )
            contador_correlacion += 1

        # Agregar siempre la columna a df_mapeo_fusion
        df_mapeo_fusion = pd.concat(
            [df_mapeo_fusion, pd.DataFrame([{ "ORIGINAL": columna, "SAP": columna_en_df2 }])],
            ignore_index=True
        )

    # Calculo de efectividad
    porcentaje_exito = round(macheo_exitoso / df1.shape[1] * 100, 2)

    return (
        df_fusionado,
        df_no_correlacionado,
        list(set(df2.columns) - columnas_utilizadas),
        list(columnas_utilizadas),
        columnas_utilizadas_multiples_veces,
        macheo_exitoso,
        porcentaje_exito,
        mode_col_name_matching,
        mode_matching,
        values_matching,
        similarity_matching,
        df_mapeo_fusion
    )


# Comparamos el 100% con modas distintas

## Ejecucion codigo df al 100%

In [24]:
# Parametros opcionales
realizar_preprocesado = False
utilizar_columnas_ya_utilizadas=True

# Ejecutar la función actualizada
df_fusionado, df_no_correlacionado, columnas_no_utilizadas, \
columnas_utilizadas, columnas_utilizadas_multiples_veces, \
macheo_exitoso, porcentaje_exito, mode_col_name_matching, \
mode_matching, values_matching, similarity_matching, df_mapeo_fusion = \
macheo_columnas_df(
    df1, df2,
    realizar_preprocesado=realizar_preprocesado,
    utilizar_columnas_ya_utilizadas=utilizar_columnas_ya_utilizadas
)

# Calcular el total de reutilizaciones de columnas
reutilizacion_total = sum(columnas_utilizadas_multiples_veces.values())

# Visualizar resultados
print("Resultados del macheo:")
print("----------------------------------------")
print(f"Número total de macheos exitosos: {macheo_exitoso}")
print(f"Porcentaje de éxito: {porcentaje_exito}%")
print("Columnas no utilizadas:", columnas_no_utilizadas)
print("Columnas utilizadas (con número de usos):", columnas_utilizadas)
print("Columnas utilizadas múltiples veces:", columnas_utilizadas_multiples_veces)
print(f"Reutilización total de columnas: {reutilizacion_total}")
print(f"Coincidencias combinadas (moda + nombre): {mode_col_name_matching}")
print(f"Coincidencias por moda: {mode_matching}")
print(f"Coincidencias por valores: {values_matching}")
print(f"Coincidencias por nombre de columna: {similarity_matching}")
print("----------------------------------------")

display(df_fusionado)
display(df_no_correlacionado)
display(df_mapeo_fusion)

🔍 Analizando columna de df1: 'Ejercicio'
🔍 Analizando columna de df1: 'Período'
✅ Mejor coincidencia para 'Período' ('Period') → 'PERIOD_ID' (80.0%)
✅ Mejor coincidencia para 'Período' ('Period') → 'PERIOD_ID' (80.0%)
🔍 Analizando columna de df1: 'Cliente'
✅ Mejor coincidencia para 'Cliente' ('Customer') → 'CUSTOMER_ID' (84.21%)
🔗 Mejor macheo por modas para la columna 'Cliente' es 'CUSTOMER_ID' con 19/20 coincidencias.
🔍 Analizando columna de df1: 'Grupo de clientes'
✅ Mejor coincidencia para 'Grupo de clientes' ('Customer group') → 'CUSTOMER_GROUP_ID' (90.32%)
✅ Macheo perfecto de modas (20/20 coincidencias) con la columna 'CUSTOMER_GROUP_ID'.
🔗 Mejor macheo por modas para la columna 'Grupo de clientes' es 'CUSTOMER_GROUP_ID' con 20/20 coincidencias.
🔍 Analizando columna de df1: 'Oficina de ventas'
✅ Mejor coincidencia para 'Oficina de ventas' ('Sales office') → 'SALES_OFFICE_ID' (88.89%)
✅ Macheo perfecto de modas (20/20 coincidencias) con la columna 'SALES_OFFICE_ID'.
🔗 Mejor mache

,Ejercicio,buscar_correlacion_sap_1,Período,PERIOD_ID_similarity_col_name,Cliente,CUSTOMER_ID_mode_col_name,Grupo de clientes,CUSTOMER_GROUP_ID_mode_col_name,Oficina de ventas,SALES_OFFICE_ID_mode_col_name,Canal distribución,INDUSTRY_ID_mode,Centro de beneficio,CEPCT_PRACTICE_ID_mode,Organización ventas,COMPANY_CODE_ID_mode,Sector,DIVISION_ID_mode,CECO Origen,PRACTICE_COST_CENTER_ID_mode,Venta extraordinaria,EXTRAORDINARY_SALE_ID_similarity_col_name,Cat.centro de coste,CCCATEGORY_ID_values,Clase de operación,RECORD_TYPE_ID_mode,Clase de proyecto,PROJECT_TYPE_ID_mode,Clv.periodificación,RA_ID_mode,ID Comercial,VASS_SALES_COMMERCIAL_ID_mode,Elemento PEP,buscar_correlacion_sap_2,Período/Año,PERIOD_YEAR_similarity_col_name,Doble Booking,buscar_correlacion_sap_3,Sociedad,COMPANY_CODE_ID_mode,Proyecto estratégico,STRATEGY_PROJECT_ID_similarity_col_name,ID Comercial.1,APPLICANT_NO_TEXT_mode,Área funcional,CLAN_ID_mode,Período.1,PERIOD_ID_similarity_col_name,Cliente.1,CUSTOMER_TEXT_mode,Grupo clientes,CUSTOMER_GROUP_TEXT_mode_col_name,Canal distrib.,INDUSTRY_TEXT_mode,CeBe,buscar_correlacion_sap_4,Organiz.ventas,SALES_OFFICE_TEXT_mode,Sector.1,DIVISION_TEXT_mode,CECO Origen.1,buscar_correlacion_sap_5,Venta ext.,buscar_correlacion_sap_6,Clase proyecto,PROJECT_TYPE_TEXT_values,Clase operación,RECORD_TYPE_TEXT_mode,Clv.periodif.,RA_TEXT_values,Elemento PEP.1,buscar_correlacion_sap_7,Período/Año.1,PERIOD_YEAR_similarity_col_name,Sociedad.1,SALES_OFFICE_TEXT_mode,Área funcional.1,buscar_correlacion_sap_8,Categoría CeCo,CCCATEGORY_TEXT_values,Inducido,INDUCED_ID_similarity_col_name,Comercial de VASS,VASS_SALES_COMMERCIAL_ID_mode_col_name,Intercompany,INTERCOMPANY_ID_similarity_col_name,A,CALC_A_mode,A.1,buscar_correlacion_sap_9,A.3,CALC_A_mode,B,CALC_A_mode,B.1,B15_similarity_col_name,B.1.1,BB11_similarity_col_name,B.1.2,buscar_correlacion_sap_10,B.1.3,buscar_correlacion_sap_11,B.1.5,B15_similarity_col_name,B.1.6,B16_similarity_col_name,B.1.7,B17_similarity_col_name,B.1.8,B18_similarity_col_name,B.1.9,B19_similarity_col_name,B.1.10,BB11_similarity_col_name,B.1.11,B111_mode_col_name,B.1.4.0,B110_similarity_col_name,B.1.12,BB11_similarity_col_name,B.2.1,buscar_correlacion_sap_12,B.2.2,buscar_correlacion_sap_13,B.2.3,buscar_correlacion_sap_14,B.3,buscar_correlacion_sap_15,B.3.1,CALC_A_mode,B.3.2,CALC_A_mode,B.4,buscar_correlacion_sap_16,B.5,B15_similarity_col_name,B.6,B16_similarity_col_name,B.7,B17_similarity_col_name,CostesIO,buscar_correlacion_sap_17,Total IO,CALC_B1_mode,T.1,CALC_A_mode,T.1.1,DIVISION_ID_mode,T.2,CALC_A_mode,T.2.1,DIVISION_ID_mode,B.B,CALC_B_mode,B.B.1,BB11_similarity_col_name,B.B.1.1,BB11_similarity_col_name,B.B.2,CALC_B_mode,MARGEN SOF,CALC_A_mode,% Mg.Comer,DIVISION_ID_mode,B.C,CALC_B_mode,B.C.1,CALC_B_mode,B.C.2,CALC_B_mode,B.C3,CALC_B_mode,B.C.4,buscar_correlacion_sap_18,B.C.5,buscar_correlacion_sap_19,T.3,CALC_A_mode,% Mg.Softw,DIVISION_ID_mode,C,CALC_A_mode,C.1,CALC_A_mode,C.2,CALC_A_mode,C.3,CALC_B_mode,C.4,buscar_correlacion_sap_20,C.5,CALC_A_mode,C.6,buscar_correlacion_sap_21,C.7,buscar_correlacion_sap_22,C.8,buscar_correlacion_sap_23,C.9,buscar_correlacion_sap_24,T.4,CALC_A_mode,T.4%,DIVISION_ID_mode,D,CALC_A_mode,D.1,CALC_A_mode,D.2,buscar_correlacion_sap_25,T.5,CALC_A_mode,T.5.1,DIVISION_ID_mode,E,CALC_A_mode,E.1,buscar_correlacion_sap_26,E.2,CALC_A_mode,E.3,buscar_correlacion_sap_27,T.6,CALC_A_mode,T.6.1,DIVISION_ID_mode,OtrosGSala,buscar_correlacion_sap_28,X.1,buscar_correlacion_sap_29,T.7,CALC_A_mode,CostPonder,buscar_correlacion_sap_30,Sum IngDir,CALC_A_mode,B.1.4,CALC_B32_mode,.,buscar_correlacion_sap_31,..1,buscar_correlacion_sap_32,C.3.1,CALC_A_mode,C.Hor.Estr,buscar_correlacion_sap_33,Dto 4% Tel,buscar_correlacion_sap_34,ImpProyect,buscar_correlacion_sap_35,GuardiasPr,buscar_correlacion_sap_36,Oficina ventas,SALES_OFFICE_TEXT_mode_col_name,BDI,CALC_A_mode,Moneda,CURRENCY_similarity_col_name,Gast.Pr.In,CALC_A_mode,Prev.exter,buscar_correlacion_sap_37,Region,REGION_ID_mode_col_name,GEO,GEO_ID_mode,Pais d

,Ejercicio,Elemento PEP,Doble Booking,CeBe,CECO Origen.1,Venta ext.,Elemento PEP.1,Área funcional.1,A.1,B.1.2,B.1.3,B.2.1,B.2.2,B.2.3,B.3,B.4,CostesIO,B.C.4,B.C.5,C.4,C.6,C.7,C.8,C.9,D.2,E.1,E.3,OtrosGSala,X.1,CostPonder,.,..1,C.Hor.Estr,Dto 4% Tel,ImpProyect,GuardiasPr,Prev.exter,GEO.1,Cluster,Cluster.1,E.4,DATA&IA&CLOUD,DATA&IA&CLOUD.1,Digital Customer Experience,Digital Custo. Expe.,Financial Service,Financial Service.1,NATEEVO,NATEEVO.1,SAP,SAP.1
0,2025,NaN,0,CEBE SIN ASIGNAR,Sin asignar,No,Sin asignar,Sin asignar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,SPAIN,NaN,Sin asignar,0.0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False
1,2025,NaN,0,CEBE SIN ASIGNAR,Sin asignar,No,Sin asignar,Sin asignar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,SPAIN,NaN,Sin asignar,0.0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False
2,2025,NaN,0,CEBE SIN ASIGNAR,Sin asignar,No,Sin asignar,Sin asignar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,SPAIN,NaN,Sin asignar,0.0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False
3,2025,NaN,0,CEBE SIN ASIGNAR,Sin asignar,No,Sin asignar,Sin asignar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,SPAIN,NaN,Sin asignar,0.0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False
4,2025,NaN,0,CEBE SIN ASIGNAR,Sin asignar,No,Sin asignar,Sin asignar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,SPAIN,NaN,Sin asignar,0.0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22581,2025,NaN,0,SAP TECHNOLOGY,SAP TECHNOLOGY,No,Sin asignar,SAP CX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,EMEAP,NaN,Sin asignar,0.0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False
22582,2025,NaN,0,SAP TECHNOLOGY,SAP TECHNOLOGY,No,Sin asignar,SAP CX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-16.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,EMEAP,NaN,Sin asignar,0.0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False
22583,2025,NaN,0,SAP DELIVERY SUCCESS,DSO,No,Sin asignar,SAP CX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1392.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,EMEAP,NaN,Sin asignar,0.0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False
22584,2025,NaN,0,SAP DELIVERY SUCCESS,DSO,No,Sin asignar,SAP CX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,EMEAP,NaN,Sin asignar,0.0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False


,ORIGINAL,SAP
0,Ejercicio,buscar_correlacion_sap_1
1,Período,PERIOD_ID
2,Cliente,CUSTOMER_ID
3,Grupo de clientes,CUSTOMER_GROUP_ID
4,Oficina de ventas,SALES_OFFICE_ID
...,...,...
149,Financial Service.1,buscar_correlacion_sap_47
150,NATEEVO,buscar_correlacion_sap_48
151,NATEEVO.1,buscar_correlacion_sap_49
152,SAP,buscar_correlacion_sap_50


# Creo arhivos definitivos

In [38]:
# Creo la carpeta 'mapeo' si no existe
output_folder = "mapeo"
os.makedirs(output_folder, exist_ok=True)

# Defino la ruta del archivo Excel
output_path = os.path.join(output_folder, "mapeo_resultados.xlsx")

# Guardo los DataFrames en un archivo Excel con dos hojas
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    df_fusionado.to_excel(writer, sheet_name="Fusionado", index=False)
    df_mapeo_fusion.to_excel(writer, sheet_name="Mapeo", index=False)

print(f"✅ Archivo guardado en: {output_path}")


✅ Archivo guardado en: mapeo\df_sap.xlsx
